## Εθνικό Μετσόβιο Πολυτεχνείο 
## Σχολή Ηλεκτρολόγων Μηχανικών & Μηχανικών Υπολογιστών


### Μάθημα: Στοχαστικές διαδικασίες
### Διδάσκων: Μιχαήλ Λουλάκης
### Ακαδημαϊκό έτος: 2017-2018


### Ιωάννης Κ. Γεωργακόπουλος
### Α.Μ.: 03111512


# 2η Εργαστηριακή Άσκηση

Σε αυτή την άσκηση θα εξοικοιωθούμε με την μέθοδο Mode Carlo.

## Ασκήσεις 33-35 βιβλίου:

Στο πρώτο μέρος της άσκησης θα επιλύσουμε την άσκηση 19 του βιβλίου, θέτοντας $p = \frac{1}{6}$. 


**Άσκηση 19** Ο πίνακας πιθανοτήτων μετάβασης P, της μαρκοβιανής αλυσίδας με χώρο καταστάσεων  $\mathbb{X}=$ {0,1,2} γράφεται για $p = \frac{1}{6}$: 

$P=\begin{pmatrix}0&1/2&1/2\\1/3&1/3&1/3\\1/6&1/2&1/3\end{pmatrix}$ 

Σημείωση: Στην εκφώνηση του βιβλίου, ο χώρος καταστάσεων είναι ο $\mathbb{X}=$ {1,2,3}, αλλά εμείς τον τροποιούμε σε $\mathbb{X}=$ {0,1,2} χάριν ευκολίας.



 ### Κατασκευή αρχικής κατανομής & πίνακα πιθανοτήτων μετάβασης


Παρακάτω κατασκευάζουμε την αρχική κατανομή, τον πίνακα πιθανοτήτων μετάβασης, και θέτουμε $p=1/6$.

Χρησιμοποιούμε την random.seed() για την εποπτεία των αποτελεσμάτων του κώδικά μας. 
(H random τροφοδοτείται με την ίδια αρχική τιμή κάθε φορα, οπότε η ακολουθία αριθμών που παράγει παραμένει σταθερή.)


In [1]:
import random
random.seed(2018)  # for reproducibility
from simple_markov_chain_lib import markov_chain

p = 1/6

# A dictionary for the initial distibution. 
# We prescribe the initial distribution
init_probs = {0: 1.0} 
 
# A dictionary for the transition probability  matrix. 
# Every state-key corresponds to a list with tuples of (Next_State,Probability) 
markov_table = {
    0: {1: 1/2, 2: 1/2},
    1: {0: 1/3, 1: 1/3, 2: 1/3},
    2: {0: p, 1: 2/3-p, 2: 1/3}
}
 
# Ok... we are ready now
# Let's construct a Markov Chain. So let's call the constructor
mc = markov_chain(markov_table, init_probs)

Θέλουμε να εκτιμήσουμε την πιθανότητα $\mathbb{P}_0[X_{20}=0]=\mathbb{P}\left[X_{20} = 0 \mid X_0 = 0 \right]$. 

Για να το κάνουμε αυτό, θα τρέξουμε την αλυσίδα πολλές φορές (παράμετρος `N`) ξεκινώντας πάντα από την κατάσταση `0` και θα μετρήσουμε σε πόσες από αυτές η αλυσίδα βρέθηκε στην κατάσταση `0` μετά από `20` βήματα (παράμετρος `steps`), δηλαδή πόσες φορές συνέβη το ενδεχόμενο $\{X_{20}=0\}$. Κάθε φορά που εκτελούμε τον βρόχο **`for`** παρακάτω παίρνουμε ένα ανεξάρτητο δείγμα της τ.μ. $Y$ που έχει την τιμή 1, αν $X_{20}=0$, και την τιμή 0, αν $X_{20}\neq 0$. Ο νόμος των μεγάλων αριθμών δίνει ότι: 
$$ \lim_{N\to\infty}\frac{Y_1+\cdots+Y_N}{N}=\mathbb{E}_0[Y]=\mathbb{P}_0[X_{20}=0].$$

Επομένως, όταν το `N` είναι μεγάλο, το ποσοστό των φορών στις οποίες συνέβη το ενδεχόμενο $\{X_{20}=0\}$ είναι μια καλή εκτίμηση της πιθανότητας $\mathbb{P}_0[X_{20}=0]$. Το ποσοστό αυτό $\hat{p}_N$ που ονομάζεται `phat` στον παρακάτω κώδικα είναι η εκτιμήτρια Monte Carlo της πιθανότητας $\mathbb{P}_0[X_{20}=0]$.

Παρακάτω έχουμε κατασκευάσει μία συνάρτηση Monte Carlo που να υπολογίζει το phat, δεχόμενη ως εισόδους το πλήθος επανήψεων Ν και τα βήματα steps. Το print_results το χρησιμοποιούμε εδώ για να τεστάρουμεαποτελέσματα.

In [2]:
def Monte_Carlo(N, steps, print_results):  ## N, steps: Experiment parameters. print_results: wether to print results.
    ## Simulation
    counter = 0  # to count the number of times the event {X_20  = 0} occurs
    for i in range(N): 
        mc.start()  # new experiment
        for j in range(steps):  mc.move()
        if mc.running_state == 0:  counter += 1
        phat = counter / N
    if print_results == 1:
        print(   
           """
           We executed {0} times the first {1} steps of the markov chain
           and we captured the running state in state zero {2} times.
           So we estimate the Pr[X_{1} = 0 | X_0 = 0] to be {3}
           """.format(N, steps, counter, phat)
        )
    return phat



Monte_Carlo(2000, 20, 1) #Παράδειγμα χρησιμοποίησής της func που κατασκευάσαμε, με εκτύπωση των αποτελεσμάτων.




           We executed 2000 times the first 20 steps of the markov chain
           and we captured the running state in state zero 417 times.
           So we estimate the Pr[X_20 = 0 | X_0 = 0] to be 0.2085
           


0.2085

Το επόμενο κομμάτι κώδικα δημιουργεί μια λίστα $Μ=50$ δειγμάτων από εκτιμήτριες Monte Carlo $\hat{p}_N$ και υπολογίζει τη μέση τιμή και τη διασπορά, για N=200 και Ν=20.0000 αντίστοιχα.

In [3]:
import statistics as stat    ## import the library statistics. We will use it to compute the mean and variance of our list
estimates=[]                 ## create the empty list estimates.
for i in range(50):                            ## in each of 50 runs generate a Monte carlo phat, without printing. 
    estimates.append( Monte_Carlo(200, 20, 0)) ## and append it to the list estimates
print(
    """ 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=200 είναι {0:.5f} και η διακύμανση {1:.5f} 
    """.format(stat.mean(estimates), stat.variance(estimates))
)
    ## compute the mean and variance of the elements in our list keeping only 5 decimal digits
    
estimates=[]                 ## empty list estimates.
for i in range(50):                            ## in each of 50 runs generate a Monte carlo phat, without printing. 
    estimates.append( Monte_Carlo(20000, 20, 0)) ## and append it to the list estimates
print(
    """ 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=20.000 είναι {0:.5f} και η διακύμανση {1:.5f}
    """.format(stat.mean(estimates), stat.variance(estimates))
)
    ## compute the mean and variance of the elements in our list keeping only 5 decimal digits


 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=200 είναι 0.19990 και η διακύμανση 0.00082 
    
 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=20.000 είναι 0.20334 και η διακύμανση 0.00001
    


## Ερωταπαντήσεις:

### 1. Ποια είναι μέση εκτίμηση της πιθανότητας $\mathbb{P}_0[X_{20}=0]$ όταν $N=200$ και ποια όταν $N=20.000$;

Για  τιμή $Ν=200$:   $\bar{x} = 0.19990$ 

Για $Ν=200.000$:   $\bar{x} = 0.20334$ 

Παρατηρούμε ότι οι δύο τιμές διαφέρουν ελάχιστα μεταξύ τους.

### 2. Είναι αυτά τα αποτελέσματα κοντά στη θεωρητική τιμή που έχετε υπολογίσει;

 Η θεωρητική τιμή που είχαμε υπολογίσει ήταν περί του $0.2$. Άρα το αποτέλεσμα $\approx 0.2$ είναι ικανοποιητικό ως προς την θεωρητική τιμή.

### 3. Ποια είναι η δειγματική διασπορά των $M=50$ εκτιμήσεων στις δύο περιπτώσεις;

Για  τιμή $Ν=200$:   ${σ}^2 = 0.00082$ 

Για $Ν=20.000$:   ${σ}^2 = 0.00001$ 

Βλέπουμε ότι στην πρώτη περίπτωση έχουμε κάποια διασπορά, αλλά στην 2η περίπτωση, τα 4 πρώτα σημαντικά στοιχεία είναι μηδενικά, οπότε υπάρχει φαινομενικά μηδαμινή διασπορά. Οπότε αξίζει να τρέξουμε N=20.000.

### 4. Αλλάζει ουσιαστικά η εκτίμησή σας, αν η αλυσίδα ξεκινάει από την κατάσταση `2` (τρίτη γραμμή του πίνακα);

Όχι, αλλά σε περίπτωση που δε με πιστεύετε:

In [4]:
init_probs = {2: 1.0} # Αρχικοποιούμε την κατανομή στην θέση 2 (3η γραμμή, ξεκινάμε από το 0, ως γνήσια παιδιά της C)

mc = markov_chain(markov_table, init_probs) #ξαναφτιάχνουμε την αλυσίδα

import statistics as stat    ## import the library statistics. We will use it to compute the mean and variance of our list
estimates=[]                 ## create the empty list estimates.
for i in range(50):                            ## in each of 50 runs generate a Monte carlo phat, without printing. 
    estimates.append( Monte_Carlo(20000, 20, 0)) ## and append it to the list estimates
print(
    """ 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=20.000 είναι {0:.5f} και η διακύμανση {1:.5f} 
    """.format(stat.mean(estimates), stat.variance(estimates))
)
    ## compute the mean and variance of the elements in our list keeping only 5 decimal digits



 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=20.000 είναι 0.20406 και η διακύμανση 0.00001 
    


Άρα δεν υπάρχει ουσιαστική διαφορά μεταξύ των εκτιμήσεων.

## Άσκηση 38 βιβλίου:

Στην εν λόγω άσκηση, θα εκτιμήσουμε με *Monte Carlo* την πιθανότητα νίκης του παίκτη που σερβίρει (`Player A`) σε ένα παιχνίδι τέννις, αν η πιθανότητα που έχει να κερδίσει κάθε πόντο είναι $p = 0,6$. 

Θα εργαστούμε βάσει του παρακάτω διαγράμματος, το οποίο δείχνει τις μεταβάσεις ενός παιχνιδιού τέννις.

![Markov Chain of Tennis Game](https://www.wolfram.com/mathematica/new-in-9/markov-chains-and-queues/HTMLImages.en/analyze-a-tennis-game/O_23.png)

Το παραπάνω πρακτικά αποτελεί μία αλυσίδα Markov, η οποία έχει χώρο καταστάσεων:

$\mathbb{X}=$ {$0-0, 15-0, 0-15, 30-0, 15-15, 0-30, 40-0, 30-15, 15-30, 0-40, 40-15, 15-40, Deuce, AdvA, AdvB, GamA, GameB$}

Η αρχική κατανομή της προφανώς λαμβάνεται, δίνοντας πιθανότητα $1$ στην κατάσταση $0-0$, και ο πίνακας μεταβάσεών της, μετατρέποντας τον παραπάνω γράφο σε πίνακα, δίνοντας πιθανότητα $p$ σε κάθε πόντο του Α, και $1-p$, σε κάθε πόντο του B.

Παρακάτω κατασκευάζουμε στην python, την αρχική κατανομή και τον πίνακα μεταβάσεων.

In [5]:
import random
random.seed(2018)  # for reproducibility
from simple_markov_chain_lib import markov_chain

p = 0.6

# A dictionary for the initial distibution. 
# We prescribe the initial distribution
init_probs = {' 0 -  0': 1.0} 
 
# A dictionary for the transition probability  matrix. 
# Every state-key corresponds to a list with tuples of (Next_State,Probability) 
markov_table = {
    ' 0 -  0': {'15 -  0': p, ' 0 - 15': 1-p},
    ' 0 - 15': {'15 - 15': p, ' 0 - 30': 1-p},
    ' 0 - 30': {'15 - 30': p, ' 0 - 40': 1-p},
    ' 0 - 40': {'15 - 40': p, ' GameB ': 1-p},
    '15 -  0': {'30 -  0': p, '15 - 15': 1-p},
    '15 - 15': {'30 - 15': p, '15 - 30': 1-p},
    '15 - 30': {' Deuce ': p, '15 - 40': 1-p},
    '15 - 40': {'  AdvB ': p, ' GameB ': 1-p},
    '30 -  0': {'40 -  0': p, '30 - 15': 1-p},
    '30 - 15': {'40 - 15': p, ' Deuce ': 1-p},
    '40 -  0': {' GameA ': p, '40 - 15': 1-p},
    '40 - 15': {' GameA ': p, '  AdvA ': 1-p},
    ' Deuce ': {'  AdvA ': p, '  AdvB ': 1-p},
    '  AdvA ': {' GameA ': p, ' Deuce ': 1-p},
    '  AdvB ': {' Deuce ': p, ' GameB ': 1-p},
    ' GameA ': {' GameA ': 1},
    ' GameB ': {' GameB ': 1}
}
 
# Ok... we are ready now
# Let's construct a Markov Chain. So let's call the constructor
mc = markov_chain(markov_table, init_probs)

Θέλουμε να εκτιμήσουμε την πιθανότητα $\mathbb{P}[X=GameA]=\mathbb{P}\left[X = GameA \mid X_0 = 0-0 \right]$. 

Ακολουθεί η συνάρτηση Monte_Carlo_Tennis. Σημειώνεται ότι αυτή τη φορά, η επανάληψη λήγει μόλις φτάσω σε GameA ή GameΒ (Δεν υπάρχουν προκαθορισμένα steps).

Υπόψιν, ότι παρόλο που έχουμε το while για να τερματίσουμε απευθείας την επανάληψη σε περίπτωση που φτάσουμε σε καταβόθρα, έχουμε και ορίσει και τα steps, ώστε να τερματίζει ούτως ή άλλως, και να θεωρείται Monte Carlo και όχι Las Vegas (παρόλο που η πιθανότητα να μην τερματίσει ποτέ είναι πρακτικά μηδαμινή).

In [6]:
def Monte_Carlo_Tennis(N, steps, print_results): 
    ## Simulation
    counter = 0  # to count the number of times the event {X= GameA} occurs
    for i in range(N):
        mc.start()         # new experiment
        rem_steps = steps  #βήματα που απομένουν
        while (((mc.running_state != ' GameA ') and (mc.running_state != ' GameB ')) and rem_steps > 0):  
            mc.move()
            rem_steps -= 1
        if mc.running_state == ' GameA ':  counter += 1

    phat = counter / N
    if print_results == 1: #Εαν θέλω να έχω εποπτεία στα αποτελέσματα
        print(
            """
            We executed the markov chain {0} times  
            and we captured the running state in state ' GameA ' {1} times.
            So we estimate the Pr[X_{1} = GameA | X_0 =0-0] to be {2}
            """.format(N, counter, phat)
        )
    return phat

Monte_Carlo_Tennis(200, 100, 1) #Το τρέχω μια φορά για να δω τι βγάζει


            We executed the markov chain 200 times  
            and we captured the running state in state ' GameA ' 148 times.
            So we estimate the Pr[X_148 = GameA | X_0 =0-0] to be 0.74
            


0.74

Βρήκαμε μία τιμή για την εκτιμήτρια $\hat{p}_N$, αλλά ας μην σταθούμε εδώ.

Εμπνευσμένοι από το προηγούμενο παράδειγμα, θα βρούμε ένα πλήθος Ν, για δείγμα $M=50$ εκτιμήσεων Monte Carlo, για το οποίο μηδενίζεται η διασπορά $σ^2$ στο αντίστοιχο δείγμα 50 εκτιμητριών Monte Carlo, ωστέ να είμαστε ευχαριστημένοι από την ακρίβεια.

Ακολουθεί ο κώδικας για $N=20.000$

In [7]:
import statistics as stat    ## import the library statistics. We will use it to compute the mean and variance of our list
estimates=[]                 ## create the empty list estimates.
for i in range(50):                            ## in each of 50 runs generate a Monte carlo phat, without printing. 
    estimates.append( Monte_Carlo_Tennis(20000, 100, 0)) ## and append it to the list estimates
print(
    """ 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=20.000 είναι {0:.5f} και η διακύμανση {1:.5f} 
    """.format(stat.mean(estimates), stat.variance(estimates))
)
    ## compute the mean and variance of the elements in our list keeping only 5 decimal digits

 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=20.000 είναι 0.73535 και η διακύμανση 0.00001 
    


Aς δοκιμάσουμε ξανά για $Ν=40.000$.

In [8]:
import statistics as stat    ## import the library statistics. We will use it to compute the mean and variance of our list
estimates=[]                 ## create the empty list estimates.
for i in range(50):                            ## in each of 50 runs generate a Monte carlo phat, without printing. 
    estimates.append( Monte_Carlo_Tennis(40000, 100, 0)) ## and append it to the list estimates
print(
    """ 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=40.000 είναι {0:.5f} και η διακύμανση {1:.5f} 
    """.format(stat.mean(estimates), stat.variance(estimates))
)
    ## compute the mean and variance of the elements in our list keeping only 5 decimal digits

 
    Η μέση τιμή 50 εκτιμήσεων Μόντε Κάρλο για N=40.000 είναι 0.73584 και η διακύμανση 0.00001 
    


Οπότε μπορούμε να πούμε ότι είμαστε ευχαριστημένοι με την (μέση) τιμή $\bar{\hat{p}}_Ν= 0.73584$, η οποία συνάδει και με τη θεωρητική επίλυση του προβλήματος. 